In [5]:
from pathlib import Path
from fastf1 import Cache

NB_DIR = Path.cwd().resolve()

# Try to locate the "Working directory" root by searching upwards
PROJECT_ROOT = None
for p in [NB_DIR] + list(NB_DIR.parents):
    if (p / "data_raw").exists() and (p / "notebooks").exists():
        PROJECT_ROOT = p
        break

if PROJECT_ROOT is None:
    raise RuntimeError(f"Could not find project root from {NB_DIR}. Expected folders: data_raw/ and notebooks/.")

CACHE_DIR = PROJECT_ROOT / "data_raw" / "f1_cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

Cache.enable_cache(str(CACHE_DIR))

print("CWD:", NB_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("FastF1 cache directory:", CACHE_DIR)

#load the necessary dependencies

CWD: C:\Users\aadha\OneDrive - Singapore Management University\Y2\S2\New folder\DAP\DAP F1 work\Working directory\notebooks\Data Collection
PROJECT_ROOT: C:\Users\aadha\OneDrive - Singapore Management University\Y2\S2\New folder\DAP\DAP F1 work\Working directory
FastF1 cache directory: C:\Users\aadha\OneDrive - Singapore Management University\Y2\S2\New folder\DAP\DAP F1 work\Working directory\data_raw\f1_cache


In [2]:
#All races that i can fetch

year = 2024
schedule = fastf1.get_event_schedule(year)

schedule[["RoundNumber", "EventName", "Country", "Location"]]

req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) C:\Users\aadha\AppData\Local\Temp\fastf1


,RoundNumber,EventName,Country,Location
0,0,Pre-Season Testing,Bahrain,Sakhir
1,1,Bahrain Grand Prix,Bahrain,Sakhir
2,2,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah
3,3,Australian Grand Prix,Australia,Melbourne
4,4,Japanese Grand Prix,Japan,Suzuka
5,5,Chinese Grand Prix,China,Shanghai
6,6,Miami Grand Prix,United States,Miami
7,7,Emilia Romagna Grand Prix,Italy,Imola
8,8,Monaco Grand Prix,Monaco,Monaco
9,9,Canadian Grand Prix,Canada,Montréal


###Singapore###


In [6]:
YEAR_START = 2018
YEAR_END   = 2025
COUNTRY    = "Singapore"
SESSION    = "Q"

def build_country_rounds(country: str, year_start: int, year_end: int) -> dict:
    rounds = {}
    for y in range(year_start, year_end + 1):
        try:
            sched = fastf1.get_event_schedule(y)
            row = sched[sched["Country"] == country]
            rounds[y] = None if row.empty else int(row.iloc[0]["RoundNumber"])
        except Exception:
            rounds[y] = None
    return rounds

SG_ROUNDS = build_country_rounds(COUNTRY, YEAR_START, YEAR_END)
SG_ROUNDS

{2018: 15,
 2019: 15,
 2020: None,
 2021: None,
 2022: 17,
 2023: 15,
 2024: 18,
 2025: 18}

In [7]:
failures = []
success = []

for y in range(YEAR_START, YEAR_END + 1):
    rnd = SG_ROUNDS.get(y)
    if rnd is None:
        print(f"[SKIP] {y}: {COUNTRY} not held")
        continue

    try:
        ses = fastf1.get_session(y, rnd, SESSION)
        ses.load(laps=True, telemetry=False, weather=False, messages=False)
        success.append((y, rnd, ses.event['EventName']))
        print(f"[OK] {y} R{rnd:02d} {ses.event['EventName']}")
    except Exception as e:
        failures.append((y, rnd, type(e).__name__, str(e)))
        print(f"[FAIL] {y} R{rnd:02d} -> {type(e).__name__}: {e}")

print("\nSuccess:", len(success), "Failures:", len(failures))
pd.DataFrame(success, columns=["year", "round", "event_name"])

core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '5', '77', '7', '3', '11', '8', '31', '27', '14', '55', '16', '9', '10', '20', '28', '2', '35', '18']
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


[OK] 2018 R15 Singapore Grand Prix


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['16', '44', '5', '33', '77', '23', '55', '27', '4', '11', '99', '10', '7', '20', '26', '18', '8', '63', '88', '3']
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


[OK] 2019 R15 Singapore Grand Prix
[SKIP] 2020: Singapore not held
[SKIP] 2021: Singapore not held


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '44', '55', '14', '4', '10', '1', '20', '22', '63', '18', '47', '5', '24', '77', '3', '31', '23', '6']
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '44', '63', '81', '27', '14', '22', '16', '55', '23', '43', '11', '20', '31', '3', '18', '10', '77', '24']
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


[OK] 2024 R18 Singapore Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

[OK] 2025 R18 Singapore Grand Prix

Success: 6 Failures: 0


,year,round,event_name
0,2018,15,Singapore Grand Prix
1,2019,15,Singapore Grand Prix
2,2022,17,Singapore Grand Prix
3,2023,15,Singapore Grand Prix
4,2024,18,Singapore Grand Prix
5,2025,18,Singapore Grand Prix


In [8]:
print("Cache dir:", CACHE_DIR)
print("Top-level cache contents:")
for p in sorted(CACHE_DIR.iterdir()):
    print(" -", p.name)

Cache dir: C:\Users\aadha\OneDrive - Singapore Management University\Y2\S2\New folder\DAP\DAP F1 work\Working directory\data_raw\f1_cache
Top-level cache contents:
 - 2018
 - 2019
 - 2020
 - 2021
 - 2022
 - 2023
 - 2024
 - 2025
 - fastf1_http_cache.sqlite
